In [1]:
import sys
sys.path.append(r"../../utils")
import numpy as np 
import pandas as pd
from questions_columns import sci_af_ca, suicidal_behavior, c_ssrs
from utils import simple_eda, impute_from_column

c_ssrs = c_ssrs[:-2] 

In [2]:
info_columns = ['id', 'gender', 'redcap_event_name', 'age_child_pre']
columns = set(suicidal_behavior + info_columns + sci_af_ca + c_ssrs)
id_column = 'id'
df = pd.read_csv(r'../../creating_the_clinic_dataset/preprocessed_data/merged_2021_and_2022.csv', na_values = ' ')

C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,44,50,73,84,309,437,442,443,450,457,486,502,540,549,568,728,733,745,747,752,763,801,810,829,882,883,884,885,886,887,888,889,890,891,892,951,989,994,1000,1015,1045,1065,1066,1070,1071,1072,1073,1075,1076,1077,1080,1081,1098,1103,1119,1121,1172,1174,1193,1195,1196,1197,1203,1204,1205,1206,1358,1360,1361,1362,1368,1369,1370,1371,1486,1492,1493,1496,1497,1498,1499,1508,1516,1532,1535,1546) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_intake = df[df.redcap_event_name == 'intake_arm_1'][columns]
df_pre = df[df.redcap_event_name == 'pre_treatment_arm_1'][columns]
df_5weeks = df[df.redcap_event_name == 'control_5weeks_arm_1'][columns]
df_3months = df[df.redcap_event_name == 'followup_3month_arm_1'][columns]

In [4]:
columns.remove('id')
df_pre_5weeks = pd.merge(df_pre, df_5weeks, on=['id'], how='outer', suffixes=('', '_5weeks'))

for column_name in columns:
    df_pre_5weeks = impute_from_column(df_pre_5weeks, impute_to = column_name, impute_from = f'{column_name}_5weeks')

    
    
five_weeks_cols = [i for i in df_pre_5weeks.columns if i.endswith('_5weeks')]
df_pre_5weeks = df_pre_5weeks.drop(five_weeks_cols, axis=1)

In [5]:
df_time2 = pd.merge(df_pre_5weeks, df_3months, on=['id'], how='outer', suffixes=('', '_followup_3month_arm_1'))


for column_name in columns:
    df_time2 = impute_from_column(df_time2, impute_to = column_name, impute_from = f'{column_name}_followup_3month_arm_1')

    
    
followup_3month_cols = [i for i in df_time2.columns if i.endswith('_followup_3month_arm_1')]
df_time2 = df_time2.drop(followup_3month_cols, axis=1)

In [6]:
df_intake['measurement'] = 'time1'
df_time2['measurement'] = 'time2'

In [7]:
output_df = pd.concat([df_intake, df_time2])

In [8]:
output_df = impute_from_column(output_df, impute_to = 'c_ssrs_6', impute_from = 'c_ssrs_last_visit_6')
output_df.loc[output_df[~output_df[c_ssrs].isna().all(axis=1)].index, c_ssrs] = 0
output_df[suicidal_behavior] = output_df[suicidal_behavior].replace(2, 0)

output_df = output_df[suicidal_behavior + info_columns + sci_af_ca + ['measurement']]

In [9]:
output_df['chameleon_suicide_er_stu']

114    NaN
117    NaN
119    NaN
122    NaN
131    NaN
      ... 
324    NaN
325    NaN
326    NaN
327    NaN
328    0.0
Name: chameleon_suicide_er_stu, Length: 737, dtype: float64

In [10]:
output_df.to_csv("data_for_prediction_research.csv", index=False)